In [1]:
import anndata
import scanpy as sc
import scgen
import pandas as pd  
import numpy as np

font = {'family' : 'Arial',
        'size'   : 14}

Global seed set to 0
/Users/meiqiliu/miniforge3/envs/fa_base2/lib/python3.8/site-packages/flax/struct.py:132: FutureWarning: jax.tree_util.register_keypaths is deprecated, and will be removed in a future release. Please use `register_pytree_with_keys()` instead.
  jax.tree_util.register_keypaths(data_clz, keypaths)
/Users/meiqiliu/miniforge3/envs/fa_base2/lib/python3.8/site-packages/flax/struct.py:132: FutureWarning: jax.tree_util.register_keypaths is deprecated, and will be removed in a future release. Please use `register_pytree_with_keys()` instead.
  jax.tree_util.register_keypaths(data_clz, keypaths)


In [2]:
train = sc.read("./data/train.h5ad",backup_url='https://drive.google.com/uc?id=1r87vhoLLq6PXAYdmyyd89zG90eJOFYLk')

/Users/meiqiliu/miniforge3/envs/fa_base2/lib/python3.8/site-packages/anndata/compat/__init__.py:232: FutureWarning: Moving element from .uns['neighbors']['distances'] to .obsp['distances'].

This is where adjacency matrices should go now.
  warn(
/Users/meiqiliu/miniforge3/envs/fa_base2/lib/python3.8/site-packages/anndata/compat/__init__.py:232: FutureWarning: Moving element from .uns['neighbors']['connectivities'] to .obsp['connectivities'].

This is where adjacency matrices should go now.
  warn(


In [3]:
print(train)
print(train.obs['condition'].values)

AnnData object with n_obs × n_vars = 16893 × 6998
    obs: 'condition', 'n_counts', 'n_genes', 'mt_frac', 'cell_type'
    var: 'gene_symbol', 'n_cells'
    uns: 'cell_type_colors', 'condition_colors', 'neighbors'
    obsm: 'X_pca', 'X_tsne', 'X_umap'
    obsp: 'distances', 'connectivities'
['stimulated', 'stimulated', 'stimulated', 'stimulated', 'stimulated', ..., 'control', 'control', 'control', 'control', 'control']
Length: 16893
Categories (2, object): ['control', 'stimulated']


In [4]:
train[train.obs['condition']=='control'].X

<8007x6998 sparse matrix of type '<class 'numpy.float32'>'
	with 2634770 stored elements in Compressed Sparse Row format>

In [20]:
import random
import numpy as np
import pandas as pd

# data = [1, 2, 3, 4, 5]
# sample_size = 3

# sample = random.sample(data, k=sample_size)
# print(sample)
# print(sample[:2])
# x=dict.fromkeys(range(3))
# x[0]=4
# np.array(x.values())
# df = pd.DataFrame({'x':data})
# print(df['x'].sample(frac=1, replace=True))
# df['x'].sample(frac=1, replace=True)

In [6]:
import pandas as pd  
import numpy as np
def sample_obs(adata,with_replacement,size_n):

    x = adata.X
    #print(x.shape)(16893, 6998)
    n_rows=x.shape[0]
    
    df = pd.DataFrame({'x':np.arange(n_rows)})
    #shuffle the data index
    x_sample1=df['x'].sample(frac=1, replace=with_replacement).values[:size_n]
    x_sample2=df['x'].sample(frac=1, replace=with_replacement).values[:size_n]
    
    return x[x_sample1,:], x[x_sample2,:]

sample_obs(train,True,train.X.shape[0])




(<16893x6998 sparse matrix of type '<class 'numpy.float32'>'
 	with 5536840 stored elements in Compressed Sparse Row format>,
 <16893x6998 sparse matrix of type '<class 'numpy.float32'>'
 	with 5528530 stored elements in Compressed Sparse Row format>)

In [7]:
from sklearn.metrics import mean_absolute_error
from scipy import stats
from scipy.special import kl_div
from scipy.stats import entropy

def calc_pearson(adata, conditions=None):

    control = adata[adata.obs["condition"] == conditions["x"]]
    stim = adata[adata.obs["condition"] == conditions["y"]]
    #print(control.shape,stim.shape)
    n=np.minimum(control.shape[0],stim.shape[0])
    
    #print(n)
    y,_ = sample_obs(control,True,n)
    x,_ = sample_obs(stim,True,n)
    
    x_mean = np.asarray(np.mean(x, axis=0)).ravel()
    y_mean = np.asarray(np.mean(y, axis=0)).ravel()
    mean = stats.pearsonr(x_mean, y_mean).statistic
    print("mean(sample)",mean)
    
    #print("mae:",mean_absolute_error(y,x))
    #if actual value is y and predicted value is x
    a=dict.fromkeys(range(x.shape[0]))
    pearson_obs=dict.fromkeys(range(x.shape[0]))
    x=x.toarray()
    y=y.toarray()
  
    for i in range(x.shape[0]):
        #use pd
        df = pd.DataFrame({'x':x[i], 'y':y[i]})
        matrix=df.corr(method='pearson').values
        pearson_obs[i] = matrix[0,1]
        
        #use scipy
        a[i]=stats.pearsonr(x[i], y[i]).statistic
 

    
    pr=list(pearson_obs.values())
    #print('a:',np.mean(list(a.values())))
    
   
    return np.mean(pr)

conditions = {"x": "stimulated", "y": "control"}
print(calc_pearson(train,conditions))

conditions = {"x": "control", "y": "control"}
calc_pearson(train,conditions)

mean(sample) 0.8645577239753062
0.3744390971168702
mean(sample) 0.999843350823882


0.43206085434879543

In [8]:
from sklearn.metrics import mean_absolute_error
from scipy import stats
from scipy.special import kl_div
from scipy.stats import entropy

def calc_mae(adata, conditions=None):

    control = adata[adata.obs["condition"] == conditions["x"]]
    stim = adata[adata.obs["condition"] == conditions["y"]]
    #print(control.shape,stim.shape)
    n=np.minimum(control.shape[0],stim.shape[0])
    
    #print(n)
    y,_ = sample_obs(control,True,n)
    x,_ = sample_obs(stim,True,n)
    
    x_mean = np.asarray(np.mean(x, axis=0)).ravel()
    y_mean = np.asarray(np.mean(y, axis=0)).ravel()
    mean=np.mean(np.abs(y_mean - x_mean))
    print("mean(sample)",mean)
    
    #print("mae:",mean_absolute_error(y,x))
    #if actual value is y and predicted value is x
    mae_obs=dict.fromkeys(range(x.shape[0]))
  
    for i in range(x.shape[0]):
        mae_obs[i]=np.mean(np.abs(y[i] - x[i]))
 
    
    mae=list(mae_obs.values())

   
    return np.mean(mae)

conditions = {"x": "stimulated", "y": "control"}
print(calc_mae(train,conditions))

conditions = {"x": "control", "y": "control"}
calc_mae(train,conditions)

mean(sample) 0.01485903
0.064406976
mean(sample) 0.0015097761


0.061299343

In [9]:
from scipy.special import kl_div
from scipy.stats import entropy 
from scipy.special import rel_entr
from scipy.spatial import distance
#old version, per cell 
def calc_kl(adata, conditions=None):

    control = adata[adata.obs["condition"] == conditions["x"]]
    stim = adata[adata.obs["condition"] == conditions["y"]]
    #print(control.shape,stim.shape)
    n=np.minimum(control.shape[0],stim.shape[0])
    
    #print(n)
    y,_ = sample_obs(control,True,n)
    x,_ = sample_obs(stim,True,n)
    #print(x.shape,y.shape)
    x=x.toarray()
    y=y.toarray()
       
    div_obs=dict.fromkeys(range(x.shape[0]))
    a=dict.fromkeys(range(x.shape[0]))
    for i in range(x.shape[0]):
        if np.all(x[i]==0)==True:
            print("x:",x[i])
        if np.all(y[i]==0)==True:
            print("y:",y[i])
        #print(x[i])
        a[i]=distance.jensenshannon(x[i],y[i])
        div_obs[i]=entropy(x[i],y[i])
        
       
    sparse=list(a.values())
    div=list(div_obs.values())
    print(sum(sparse)/len(sparse))
    
   
    return np.mean(div)

conditions = {"x": "stimulated", "y": "control"}
print(calc_kl(train,conditions))

conditions = {"x": "control", "y": "control"}
calc_kl(train,conditions)

0.707142938462377
inf
0.6946626982940782


inf

In [10]:
from scipy.special import kl_div
from scipy.stats import entropy 
from scipy.special import rel_entr
from scipy.spatial import distance
from scipy.stats import shapiro,ttest_rel,t,ttest_ind
#distribution based metrics 
def calc_kl(adata, conditions=None):

    control = adata[adata.obs["condition"] == conditions["x"]]
    stim = adata[adata.obs["condition"] == conditions["y"]]
    #print("ctrl,stim",control.shape,stim.shape)
    n=np.minimum(control.shape[1],stim.shape[1])
    m=np.minimum(control.shape[0],stim.shape[0])
    #first sample the data by suffling observation
    y,_ = sample_obs(control,True,n)
    x,_ = sample_obs(stim,True,n)
    #subset data for sample to have same length 
    #transpose data since we compute metric across genes 
    x=x.T.toarray()
    y=y.T.toarray()
    
    #print("x,y",x.shape,y.shape)

    div_obs=dict.fromkeys(range(x.shape[0]))
    
    kl=0
    count_nan=0
    for i in range(x.shape[0]):
#         if np.all(x[i]==0)==True:
#             print("x:",x[i])
#         if np.all(y[i]==0)==True:
#             print("y:",y[i])
        sparse=distance.jensenshannon(x[i],y[i])
        if np.isnan(sparse):#nan: caused by division by 0-vector(vec of all 0)
            count_nan+=1
            div_obs[i]=0
        else:
            kl+=sparse
            div_obs[i]=entropy(x[i],y[i])
        #print(distance.jensenshannon(x[i],y[i]))
 
    div=list(div_obs.values())
    print(kl/(x.shape[0]-count_nan))
    

    return np.mean(div)

conditions = {"x": "stimulated", "y": "control"}
print(calc_kl(train,conditions))

conditions = {"x": "control", "y": "control"}
calc_kl(train,conditions)

/Users/meiqiliu/miniforge3/envs/fa_base2/lib/python3.8/site-packages/scipy/spatial/distance.py:1291: RuntimeWarning: invalid value encountered in divide
  q = q / np.sum(q, axis=axis, keepdims=True)
/Users/meiqiliu/miniforge3/envs/fa_base2/lib/python3.8/site-packages/scipy/spatial/distance.py:1290: RuntimeWarning: invalid value encountered in divide
  p = p / np.sum(p, axis=axis, keepdims=True)


0.8127722310129523
inf


/Users/meiqiliu/miniforge3/envs/fa_base2/lib/python3.8/site-packages/scipy/spatial/distance.py:1291: RuntimeWarning: invalid value encountered in divide
  q = q / np.sum(q, axis=axis, keepdims=True)
/Users/meiqiliu/miniforge3/envs/fa_base2/lib/python3.8/site-packages/scipy/spatial/distance.py:1290: RuntimeWarning: invalid value encountered in divide
  p = p / np.sum(p, axis=axis, keepdims=True)


0.8120003121053635


inf

In [11]:
import scipy.stats as stats
def t_test(adata, conditions=None):

    control = adata[adata.obs["condition"] == conditions["x"]]
    stim = adata[adata.obs["condition"] == conditions["y"]]
    n=np.minimum(control.shape[1],stim.shape[1])
    m=np.minimum(control.shape[0],stim.shape[0])

    y,_ = sample_obs(control,True,n)
    x,_ = sample_obs(stim,True,n)   
    
    x=x.T[:,:m].toarray()
    y=y.T[:,:m].toarray()
  
    t=dict.fromkeys(range(x.shape[1]))
    p=0
    count_nan=0
    for i in range(x.shape[0]):
        #print(np.var(x[i]),np.var(y[i]))
        t_stats,p_value=stats.ttest_ind(x[i],y[i], equal_var=True)
#         t_stats,p_value=ttest_rel(x[i], y[i])
        if np.isnan(t_stats):
            count_nan+=1
            t[i]=0
        else:
            t[i]=t_stats
            p=p+p_value
        
            
#         stat, p = shapiro(y[i])
#         if p > 0.05:
#             print('Data looks normally distributed')
#         else:
#             print('Data does not look normally distributed')
       
    
    print("t-test:",np.sum(list(t.values()))/(x.shape[0]-count_nan))
    print("p-value",p/(x.shape[0]-count_nan))
    
   
   
    return np.sum(list(t.values()))/(x.shape[0]-count_nan)

conditions = {"x": "stimulated", "y": "control"}
t_test(train,conditions)

conditions = {"x": "control", "y": "control"}
t_test(train,conditions)

t-test: 0.5120583079347397
p-value 0.21448338781112083
t-test: 0.1373755175959996
p-value 0.4964027414717354


0.1373755175959996

In [12]:

from sklearn import metrics


def mmd_linear(X, Y):
    """MMD using linear kernel (i.e., k(x,y) = <x,y>)
    Note that this is not the original linear MMD, only the reformulated and faster version.
    The original version is:
        def mmd_linear(X, Y):
            XX = np.dot(X, X.T)
            YY = np.dot(Y, Y.T)
            XY = np.dot(X, Y.T)
            return XX.mean() + YY.mean() - 2 * XY.mean()
    Arguments:
        X {[n_sample1, dim]} -- [X matrix]
        Y {[n_sample2, dim]} -- [Y matrix]
    Returns:
        [scalar] -- [MMD value]
    """
    delta = X.mean(0) - Y.mean(0)
    return delta.dot(delta.T)


def mmd_rbf(X, Y, gamma=1.0):
    """MMD using rbf (gaussian) kernel (i.e., k(x,y) = exp(-gamma * ||x-y||^2 / 2))
    Arguments:
        X {[n_sample1, dim]} -- [X matrix]
        Y {[n_sample2, dim]} -- [Y matrix]
    Keyword Arguments:
        gamma {float} -- [kernel parameter] (default: {1.0})
    Returns:
        [scalar] -- [MMD value]
    """
    XX = metrics.pairwise.rbf_kernel(X, X, gamma)
    YY = metrics.pairwise.rbf_kernel(Y, Y, gamma)
    XY = metrics.pairwise.rbf_kernel(X, Y, gamma)
    return XX.mean() + YY.mean() - 2 * XY.mean()


def mmd_poly(X, Y, degree=2, gamma=1, coef0=0):
    """MMD using polynomial kernel (i.e., k(x,y) = (gamma <X, Y> + coef0)^degree)
    Arguments:
        X {[n_sample1, dim]} -- [X matrix]
        Y {[n_sample2, dim]} -- [Y matrix]
    Keyword Arguments:
        degree {int} -- [degree] (default: {2})
        gamma {int} -- [gamma] (default: {1})
        coef0 {int} -- [constant item] (default: {0})
    Returns:
        [scalar] -- [MMD value]
    """
    XX = metrics.pairwise.polynomial_kernel(X, X, degree, gamma, coef0)
    YY = metrics.pairwise.polynomial_kernel(Y, Y, degree, gamma, coef0)
    XY = metrics.pairwise.polynomial_kernel(X, Y, degree, gamma, coef0)
    return XX.mean() + YY.mean() - 2 * XY.mean()

In [13]:
def mmd(adata, conditions=None):

    control = adata[adata.obs["condition"] == conditions["x"]]
    stim = adata[adata.obs["condition"] == conditions["y"]]
    n=np.minimum(control.shape[1],stim.shape[1])
    m=np.minimum(control.shape[0],stim.shape[0])

    y,_ = sample_obs(control,True,n)
    x,_ = sample_obs(stim,True,n)   
    
    x=x.T[:,:m].toarray()
    y=y.T[:,:m].toarray()
  
    print("mmd linear:",mmd_linear(x,y))
    print("mmd rbf: ",mmd_rbf(x,y))

    return None

conditions = {"x": "stimulated", "y": "control"}
mmd(train,conditions)

conditions = {"x": "control", "y": "control"}
mmd(train,conditions)

mmd linear: 0.5565193
mmd rbf:  0.00043091876432299614
mmd linear: 0.6081152
mmd rbf:  0.0002822889946401119


In [15]:
def wasserstein(adata, conditions=None):

    control = adata[adata.obs["condition"] == conditions["x"]]
    stim = adata[adata.obs["condition"] == conditions["y"]]
    n=np.minimum(control.shape[1],stim.shape[1])
    m=np.minimum(control.shape[0],stim.shape[0])

    y,_ = sample_obs(control,True,n)
    x,_ = sample_obs(stim,True,n)   
    
    x=x.T[:,:m].toarray()
    y=y.T[:,:m].toarray()
  
    d=dict.fromkeys(range(x.shape[1]))
    
    for i in range(x.shape[0]):
       
        d[i]=stats.wasserstein_distance(x[i],y[i])
         
    print(np.mean(list(d.values())))


    return np.mean(list(d.values()))

conditions = {"x": "stimulated", "y": "control"}
wasserstein(train,conditions)

conditions = {"x": "control", "y": "control"}
wasserstein(train,conditions)


0.015276400320477314
0.002034795733287141


0.002034795733287141

In [16]:
def calc_euclidean_d(adata, conditions=None):

    control = adata[adata.obs["condition"] == conditions["x"]]
    stim = adata[adata.obs["condition"] == conditions["y"]]
    #print(control.shape,stim.shape)
    n=np.minimum(control.shape[0],stim.shape[0])
    
    #print(n)
    y,_ = sample_obs(control,True,n)
    x,_ = sample_obs(stim,True,n)
    
    x_mean = np.asarray(np.mean(x, axis=0)).ravel()
    y_mean = np.asarray(np.mean(y, axis=0)).ravel()
    mean=np.linalg.norm(x_mean-y_mean)
    print("mean(sample)",mean)
    
    x=x.toarray()
    y=y.toarray()
       
    d_obs=dict.fromkeys(range(x.shape[0]))
    a=dict.fromkeys(range(x.shape[0]))
    for i in range(x.shape[0]):
        
        d_obs[i]=np.linalg.norm(x[i]-y[i])
        
    
    distance=list(d_obs.values())

   
    return np.mean(distance)

conditions = {"x": "stimulated", "y": "control"}
print(calc_euclidean_d(train,conditions))

conditions = {"x": "control", "y": "control"}
calc_euclidean_d(train,conditions)

mean(sample) 6.5710588
21.386189
mean(sample) 0.23416795


19.898565

In [17]:

def calc_r(adata, conditions=None):

    control = adata[adata.obs["condition"] == conditions["x"]]
    stim = adata[adata.obs["condition"] == conditions["y"]]
    #print(control.shape,stim.shape)
    n=np.minimum(control.shape[0],stim.shape[0])
    
    #print(n)
    y,_ = sample_obs(control,True,n)
    x,_ = sample_obs(stim,True,n)
    
    x_mean = np.asarray(np.mean(x, axis=0)).ravel()
    y_mean = np.asarray(np.mean(y, axis=0)).ravel()
    m, b, r_value, p_value, std_err = stats.linregress(x_mean, y_mean)
    print("mean(sample)",r_value**2)
    
    x=x.toarray()
    y=y.toarray()
       
    r_obs=dict.fromkeys(range(x.shape[0]))
    
    for i in range(x.shape[0]):
        m, b, r_value, p_value, std_err = stats.linregress(x[i], y[i])
        r_obs[i]=r_value**2
        
    
    rsquare=list(r_obs.values())

   
    return np.mean(rsquare)

conditions = {"x": "stimulated", "y": "control"}
print(calc_r(train,conditions))

conditions = {"x": "control", "y": "control"}
calc_r(train,conditions)

mean(sample) 0.7463048378192959
0.1525882691324689
mean(sample) 0.9996107943932976


0.2024386254743203

In [18]:
train.obs.condition.value_counts()

stimulated    8886
control       8007
Name: condition, dtype: int64

In [ ]:
def compute_mean(sample1,sample2,axis):
    return np.mean(sample1,axis),np.mean(sample2,axis)



In [ ]:
import pandas as pd  
import numpy as np
def sample(adata,with_replacement,size_n,by_obs):
    x = adata.X
    
    if by_obs==True:
        n=x.shape[0]
    else:
        n=x.shape[1]
    
    df = pd.DataFrame({'x':np.arange(n)})
    #shuffle the data index
    x_sample1=df['x'].sample(frac=1, replace=with_replacement).values[:size_n]
    x_sample2=df['x'].sample(frac=1, replace=with_replacement).values[:size_n]
    
    #compute the mean of sample 
    if by_obs==True:
        sample1=x[x_sample1,:].toarray()
        sample2=x[x_sample2,:].toarray()
        x,y=compute_mean(sample1,sample2,1)
        
    else:
        sample1=x[:,x_sample1].toarray()
        sample2=x[:,x_sample2].toarray()
        x,y=compute_mean(sample1,sample2,0)
    return x,y
    

#     x = np.asarray(np.mean(adata.X, axis=0)).ravel()# mean for each col/gene
    
#     df = pd.DataFrame({'x':x})
    
#     x_sample1=df['x'].sample(frac=0.5, replace=with_replacement, random_state=1)
#     x_sample2=df['x'].sample(frac=0.5, replace=with_replacement, random_state=2)
    
#     return x_sample1.values, x_sample2.values
